### comment out the code below to install required packages

In [1]:
# code to install all the packages we need
# %pip install numpy
# %pip install pandas
# %pip install yfinance
# %pip install requests
# %pip install beautifulsoup4
# %pip install scikit-learn
# %pip install TA-lib
# %pip install joblib
# %pip install pickle
# %pip install Riskfolio-lib


In [2]:
# Import necessary libraries
import numpy as np
import pandas as pd
import yfinance as yf
from ibapi.client import *
from ibapi.wrapper import *
from ibapi.contract import *
from ibapi.order import *
import ibapi

import requests
import bs4 as bs
import datetime
import time
import threading
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV  
import joblib

import talib as ta
from talib import MA_Type
import pickle


In [3]:
init_event = threading.Event() # init event
id_event = threading.Event()
mkt_event = threading.Event()
hist_event = threading.Event()
order_event = threading.Event()
evec_event = threading.Event()
port_event = threading.Event()
value_event = threading.Event()

In [4]:
# define the App class
class App(EClient, EWrapper):
    def __init__(self, address, port, cid):
        EClient.__init__(self, self)
        # # list to store data
        self.bar_dict = {}
        self.mkt_price = []
        # create a connection with IBKR
        self.connect(address, port, cid)
        self.last_portfolio = pd.DataFrame(columns=["ticker","quantity","marketPrice","marketValue","averageCost","unrealizedPNL","realizedPNL"])
        self.value = 0
        # start client
        thread = threading.Thread(target=self.run)
        thread.start()
        init_event.set()


    def nextValidId(self, orderId: int):
        # provide a new order id for each of my requests
        super().nextValidId(orderId)
        logging.debug("setting nextValidOrderId: %d", orderId)
        self.nextValidOrderId = orderId
        print("NextValidId:", orderId)
        id_event.set()

    def tickPrice(self, reqId: int, tickType: int, price: float, attrib: ibapi.common.TickAttrib):
            print("Tick Price. Ticker Id:", reqId, "tickType:", tickType, "Price:", price)
            if tickType == 9: # if tickType is Close Price
                self.mkt_price.append([reqId, price])
                mkt_event.set()

    def historicalData(self, reqId, bar):
        if reqId not in self.bar_dict.keys():
            self.bar_dict[reqId] = []
        self.bar_dict[reqId].append(vars(bar))
        
    def historicalDataEnd(self, reqId, start, end):
        print(f"end of historicalData")
        hist_event.set()


    # implement code to monitor trade status and receive confirmation of the trade
    def openOrder(self, orderId: OrderId, contract: Contract, order: Order, orderstate: OrderState):
        # openorder callback
        print(f"openOrder. orderId:{orderId}, contract:{contract}, order:{order}")
        order_event.set()

    def orderStatus(self, orderId: OrderId, status: str, filled: float, reamining: float, avgFillPrice: float,
                    permId: int, parenId: int, lastFillPrice: float, clientId: int, whyHeld:str, mktCapPrice: float):
                    # orderstatus callback
        print(f"orderStatus. orderId: {orderId}, status: {status}, filled: {filled}, remaining:{reamining}, avgFillPrice: {avgFillPrice}, permId:{permId}, parentId:{parenId}, lastFillPrice: {lastFillPrice}, clientId: {clientId}, whyHeld: {whyHeld}, mktCapPrice:{mktCapPrice}")

    def execDetails(self, reqId: int, contract: Contract, execution: Execution):
        print(f"execDetails. reqId: {reqId}, contract: {contract}, execution: {execution}")
        evec_event.set()
    # basically a summary

    def commissionReport(self, commissionReport: CommissionReport):
        super().commissionReport(commissionReport)
        print("CommissionReport.", commissionReport)
        
    # called when query portfolio information
    def updatePortfolio(self, contract, position, marketPrice, marketValue, averageCost, unrealizedPNL, realizedPNL, accountName):
        self.last_portfolio = pd.concat([self.last_portfolio,
                                        pd.DataFrame([[contract.symbol, position, marketPrice, marketValue, averageCost, unrealizedPNL, realizedPNL]],
                                                    columns=self.last_portfolio.columns)],
                                        ignore_index=True)
        port_event.set()

    # query total value
    def accountSummary(self, reqId: int, account: str, tag: str, value: str, currency: str):
        self.value = float(value)
        print("Total value of the account: ", self.value)
    
    def accountSummaryEnd(self, reqId: int):
        print('end of account summary')
        value_event.set()

In [5]:
# Connect to the TWS API
app = App('127.0.0.1', 7497, 1000)
init_event.wait() # wait until it's connected
init_event.clear()

In [6]:
app.reqMarketDataType(1) # in case it doesn't work, change 1 to 3

NextValidId: 65


In [7]:
with open('portfolio_list.pickle', 'rb') as f:
    portfolio_list = pickle.load(f)

print(portfolio_list)

['LNT', 'AWK', 'CHTR', 'FRC', 'MRNA', 'OGN', 'SRE', 'UNH', 'UHS', 'WBA']


# query the current portfolio


In [8]:
app.reqAccountUpdates(True, "DU6730183") #TODO: replace the DU222526 with our account ID
# port_event.wait()
# port_event.clear()

time.sleep(10)
last_portfolio = app.last_portfolio
app.reqAccountUpdates(False, "DU6730183") 

ERROR:ibapi.wrapper:ERROR -1 2100 API client has been unsubscribed from account data.


In [9]:
last_portfolio

,ticker,quantity,marketPrice,marketValue,averageCost,unrealizedPNL,realizedPNL
0,A,623,136.094131,84786.64,141.355000,-3277.52,0.0
1,AWK,635,141.679062,89966.20,138.125000,2256.83,0.0
2,AZO,35,2398.387939,83943.58,2447.168571,-1707.32,0.0
3,BXP,1369,52.674000,72110.71,63.235000,-14458.01,0.0
4,IFF,998,83.260994,83094.47,87.256002,-3987.02,0.0
5,MHK,865,96.158432,83177.04,101.935000,-4996.73,0.0
6,MOH,326,263.501404,85901.46,269.935000,-2097.35,0.0
7,MRO,3460,21.600000,74736.00,24.915000,-11469.90,0.0
8,PKI,707,125.715759,88881.04,126.475000,-536.78,0.0
9,VFC,3704,21.326000,78991.50,23.575000,-8330.30,0.0


In [10]:
app.reqAccountUpdates(False, "DU6730183")#TODO: replace the DU222526 with our account ID
app.reqAccountSummary(9001, "All", "NetLiquidation")
value_event.wait()
value_event.clear()
# time.sleep(3)
total_value = app.value
app.cancelAccountSummary(9001)

Total value of the account:  876160.17
end of account summary


In [11]:
total_value

876160.17

# implementation
* decide what to buy & sell, as well as the quantity

In [12]:

# get the market price
for i, t in enumerate(portfolio_list):
    contract = Contract()
    contract.symbol = t
    contract.secType = "STK"
    contract.currency = "USD"
    contract.exchange = "SMART"
    app.reqMktData(i, contract, "", False, False, [])
    mkt_event.wait()
    mkt_event.clear()
    app.cancelMktData(i)

mkt_df = pd.DataFrame(app.mkt_price, columns=['reqId', 'mkt_price'])
assert sum(mkt_df.isnull().any()) == 0, "didn't get all the market price"
assert len(mkt_df) == len(portfolio_list), "didn't get all the market price"


Tick Price. Ticker Id: 0 tickType: 4 Price: 53.12
Tick Price. Ticker Id: 0 tickType: 6 Price: 54.01
Tick Price. Ticker Id: 0 tickType: 7 Price: 52.8
Tick Price. Ticker Id: 0 tickType: 9 Price: 53.1
Tick Price. Ticker Id: 0 tickType: 14 Price: 52.89
Tick Price. Ticker Id: 0 tickType: 1 Price: 53.09
Tick Price. Ticker Id: 0 tickType: 2 Price: 53.11
Tick Price. Ticker Id: 1 tickType: 1 Price: 141.73
Tick Price. Ticker Id: 1 tickType: 2 Price: 141.78
Tick Price. Ticker Id: 1 tickType: 4 Price: 141.73
Tick Price. Ticker Id: 1 tickType: 6 Price: 142.97
Tick Price. Ticker Id: 1 tickType: 7 Price: 139.8
Tick Price. Ticker Id: 1 tickType: 9 Price: 140.63
Tick Price. Ticker Id: 1 tickType: 14 Price: 140.99
Tick Price. Ticker Id: 2 tickType: 4 Price: 345.85
Tick Price. Ticker Id: 2 tickType: 6 Price: 348.05
Tick Price. Ticker Id: 2 tickType: 7 Price: 337.3
Tick Price. Ticker Id: 2 tickType: 9 Price: 341.94
Tick Price. Ticker Id: 2 tickType: 14 Price: 339.16
Tick Price. Ticker Id: 2 tickType: 1 Pr

In [13]:
target_portfolio = pd.DataFrame(columns=['tickers', 'weight', 'mkt_price'])
target_portfolio['tickers'] = portfolio_list
target_portfolio['weight'] = np.repeat(1/len(portfolio_list), len(portfolio_list))
target_portfolio['mkt_price'] = mkt_df['mkt_price']

In [14]:
target_portfolio['quantity'] = np.floor(target_portfolio['weight'] * (total_value * 0.95) / target_portfolio['mkt_price'])

In [15]:
last_portfolio

,ticker,quantity,marketPrice,marketValue,averageCost,unrealizedPNL,realizedPNL
0,A,623,136.094131,84786.64,141.355000,-3277.52,0.0
1,AWK,635,141.679062,89966.20,138.125000,2256.83,0.0
2,AZO,35,2398.387939,83943.58,2447.168571,-1707.32,0.0
3,BXP,1369,52.674000,72110.71,63.235000,-14458.01,0.0
4,IFF,998,83.260994,83094.47,87.256002,-3987.02,0.0
5,MHK,865,96.158432,83177.04,101.935000,-4996.73,0.0
6,MOH,326,263.501404,85901.46,269.935000,-2097.35,0.0
7,MRO,3460,21.600000,74736.00,24.915000,-11469.90,0.0
8,PKI,707,125.715759,88881.04,126.475000,-536.78,0.0
9,VFC,3704,21.326000,78991.50,23.575000,-8330.30,0.0


In [16]:
target_portfolio

,tickers,weight,mkt_price,quantity
0,LNT,0.1,53.10,1567.0
1,AWK,0.1,140.63,591.0
2,CHTR,0.1,341.94,243.0
3,FRC,0.1,31.16,2671.0
4,MRNA,0.1,149.60,556.0
5,OGN,0.1,21.52,3867.0
6,SRE,0.1,146.36,568.0
7,UNH,0.1,465.43,178.0
8,UHS,0.1,117.65,707.0
9,WBA,0.1,33.54,2481.0


# figure out the best order of our order. (To save commission fee)

* target_portfolio
> target_portfolio python pandas dataframe containing information of stocks portfolio(target portfolio) that I want to form through trading. The first column of the dataframe is stocks' tickers and the second column of the dataframe is the quantity of each stock that I want to hold. 

* last_portfolio
> last_portfolio is a dataframe containing information of last portfolio that I hold

In [17]:
# Create a list to store the stocks that need to be sold
stocks_to_sell = []
# Create a list to store the stocks that need to buy
stocks_to_buy = []

# Iterate through the target_portfolio dataframe
for index, row in target_portfolio.iterrows():
    # Check if the stock is present in the last_portfolio dataframe
    last_portfolio_stock = last_portfolio[last_portfolio['ticker'] == row['tickers']]
    if not last_portfolio_stock.empty:
        # Compare the quantities
        last_portfolio_quantity = float(last_portfolio_stock['quantity'].values[0])
        if row['quantity'] < last_portfolio_quantity:
            # Calculate the difference in quantity
            quantity_diff = last_portfolio_quantity - row['quantity'] 
            # Add the stock to the list of stocks to sell
            stocks_to_sell.append([row['tickers'], quantity_diff])

        if row['quantity'] > last_portfolio_quantity:
            # Calculate the difference in quantity
            quantity_diff = row['quantity'] - last_portfolio_quantity 
            # Add the stock to the list of stocks to buy
            stocks_to_buy.append([row['tickers'], quantity_diff])

# we also need to sell the stocks that are in last_portfolio but not in target_portfolio
# Find the difference between the two portfolios
new_tickers_to_sell = set(last_portfolio['ticker']).difference(set(target_portfolio['tickers']))

for ticker in new_tickers_to_sell:
    # Find the quantity of the stock to sell from the last_portfolio
    quantity = last_portfolio.loc[last_portfolio['ticker'] == ticker, 'quantity'].values[0]
    # append to stocks to sell
    if quantity != 0:
        stocks_to_sell.append([ticker, quantity])

# we also need to buy the stocks that are in target_protfolio but not in last_portfolio
# Find the difference between the two portfolios
new_tickers_to_buy = set(target_portfolio['tickers']).difference(set(last_portfolio['ticker']))

for ticker in new_tickers_to_buy:
    # Find the quantity of the stock to buy from the target_portfolio
    quantity = target_portfolio.loc[target_portfolio['tickers'] == ticker, 'quantity'].values[0]
    # append to stocks to buy
    if quantity != 0:
        stocks_to_buy.append([ticker, quantity])

In [18]:
stocks_to_buy

[['FRC', 2671.0],
 ['UNH', 178.0],
 ['CHTR', 243.0],
 ['MRNA', 556.0],
 ['LNT', 1567.0],
 ['WBA', 2481.0],
 ['UHS', 707.0],
 ['SRE', 568.0],
 ['OGN', 3867.0]]

In [19]:
stocks_to_sell

[['AWK', 44.0],
 ['VFC', Decimal('3704')],
 ['AZO', Decimal('35')],
 ['MHK', Decimal('865')],
 ['A', Decimal('623')],
 ['MRO', Decimal('3460')],
 ['IFF', Decimal('998')],
 ['MOH', Decimal('326')],
 ['BXP', Decimal('1369')],
 ['PKI', Decimal('707')]]

# place order

In [20]:
app.reqIds(-1) # require a new id
id_event.wait()
id_event.clear()
orderid = app.nextValidOrderId
for stock in stocks_to_sell:
    # Replace the place holder values with the appropriate values
    ticker = stock[0]
    quantity = stock[1]
    contract = Contract()
    contract.symbol = ticker
    contract.secType = "STK"
    contract.exchange = "SMART"
    contract.currency = "USD"
    order = Order()
    order.action = "SELL"
    order.orderType = "MKT"
    order.totalQuantity = quantity
    order.tif = "GTC" # default is day order

    orderid += 1

    app.placeOrder(orderid, contract, order)
    order_event.wait()
    order_event.clear()

count = 0
for stock in stocks_to_buy:
    # Replace the place holder values with the appropriate values
    app.reqAccountUpdates(False, "DU6730183")#TODO: replace the DU222526 with our account ID
    app.reqAccountSummary(9001, "All", "SettledCash")
    value_event.wait()
    value_event.clear()
    cash_value = app.value
    if count >= len(stocks_to_buy)/2 and cash_value <= 0:
        break # stop buying when cash is below 0
    ticker = stock[0]
    quantity = stock[1]
    contract = Contract()
    contract.symbol = ticker
    contract.secType = "STK"
    contract.exchange = "SMART"
    contract.currency = "USD"
    order = Order()
    order.action = "BUY"
    order.orderType = "MKT"
    order.totalQuantity = quantity
    order.tif = "GTC" # default is day order

    orderid += 1
    app.placeOrder(orderid, contract, order)
    order_event.wait()
    order_event.clear()
    count +=1

NextValidId: 65
openOrder. orderId:66, contract:50530752,AWK,STK,,0,?,,SMART,,USD,AWK,AWK,False,,,,combo:, order:66,1000,1822203817: MKT SELL 44@141.73 GTC
orderStatus. orderId: 66, status: Submitted, filled: 0, remaining:44, avgFillPrice: 0.0, permId:1822203817, parentId:0, lastFillPrice: 0.0, clientId: 1000, whyHeld: , mktCapPrice:0.0
openOrder. orderId:67, contract:13440,VFC,STK,,0,?,,SMART,,USD,VFC,VFC,False,,,,combo:, order:67,1000,1822203818: MKT SELL 3704@21.34 GTC
orderStatus. orderId: 67, status: PreSubmitted, filled: 0, remaining:3704, avgFillPrice: 0.0, permId:1822203818, parentId:0, lastFillPrice: 0.0, clientId: 1000, whyHeld: , mktCapPrice:0.0
execDetails. reqId: -1, contract: 13440,VFC,STK,,0,,,PEARL,,USD,VFC,VFC,False,,,,combo:, execution: ExecId: 00025b46.6413710e.01.01, Time: 20230316 14:39:51 US/Eastern, Account: DU6730183, Exchange: PEARL, Side: SLD, Shares: 100, Price: 21.35, PermId: 1822203818, ClientId: 1000, OrderId: 67, Liquidation: 0, CumQty: 100, AvgPrice: 21.

In [21]:
app.disconnect()

openOrder. orderId:80, contract:5026984,LNT,STK,,0,?,,SMART,,USD,LNT,NMS,False,,,,combo:, order:80,1000,1822203831: MKT BUY 1567@0 GTC
orderStatus. orderId: 80, status: Submitted, filled: 603, remaining:964, avgFillPrice: 53.11, permId:1822203831, parentId:0, lastFillPrice: 53.11, clientId: 1000, whyHeld: , mktCapPrice:0.0
